In [103]:
import pandas as pd
import numpy as np

annotations = pd.read_csv('annotations.csv')

In [104]:
# !pip3 install tensorflow

In [105]:
annotations['image_name'] = annotations['image'].apply(lambda x: x.split('/')[-1])

In [106]:
annotations.head(3)

,image,id,street_width,annotator,annotation_id,pavement_type,pavement_quality,image_name
0,/data/upload/image0angle270.jpg,5,irrelevant_image,paulomanuelmaia@gmail.com,5,NaN,NaN,image0angle270.jpg
1,/data/upload/image11.jpg,53,double_car_or_more,paulomanuelmaia@gmail.com,53,alcatrao,mid,image11.jpg
2,/data/upload/image11angle0.jpg,54,single_car,paulomanuelmaia@gmail.com,54,alcatrao,low,image11angle0.jpg


In [107]:
annotations = pd.concat([annotations, pd.get_dummies(annotations['street_width'])], axis=1)
annotations = annotations.drop(columns=['double_car_or_more'])
# annotations['single_car'] = np.where(annotations['irrelevant_image'] == 1, np.nan, annotations['single_car'])

# remove other predictions in case image is irrelevant
#annotations['pavement_type'] = np.where(annotations['irrelevant_image'] == 1, np.nan, annotations['pavement_type'])
#annotations['pavement_quality'] = np.where(annotations['irrelevant_image'] == 1, np.nan, annotations['pavement_quality'])
#annotations['pavement_quality'] = np.where(annotations['pavement_type'] != 'alcatrao', np.nan, annotations['pavement_quality'])

quality_cols = pd.get_dummies(annotations['pavement_quality'])
quality_cols.columns = [x + '_quality' for x in quality_cols.columns]

annotations = pd.concat([annotations, quality_cols], axis=1)

annotations = pd.concat([annotations, pd.get_dummies(annotations['pavement_type'])], axis=1)

annotations = annotations.drop(columns=['annotation_id', 'pavement_type', 'pavement_quality', 'annotator', 'image'])

Rules for annotation and modeling:

- Whenever image is "irrelevant", dont label anything else
- Whenever type is different from "alcatrão", don't label street quality

In [108]:
annotations.head(3)

,id,street_width,image_name,irrelevant_image,single_car,high_quality,low_quality,mid_quality,alcatrao,paralelo,terra_batida
0,5,irrelevant_image,image0angle270.jpg,1,0,0,0,0,0,0,0
1,53,double_car_or_more,image11.jpg,0,0,0,0,1,1,0,0
2,54,single_car,image11angle0.jpg,0,1,0,1,0,1,0,0


In [ ]:
plot_images_per_class('irrelevant_image', show_positive_class_only=True)

In [ ]:
def model(
    lr=0.0001, input_shape=(224, 224, 3), base_model_trainable=False, model_name = 'irrelevant_vs_relevant'
):

    input_layer = Input(shape=input_shape, name="image")

    base_model = MobileNetV2(
        input_shape=input_shape, weights="imagenet", include_top=False, pooling="max"
    )

    if not base_model_trainable:
        base_model.trainable = False

    x = base_model(input_layer)

    x = Dense(200, activation="relu")(x)
    x = Dropout(0.5)(x)
    x = Dense(50, activation="relu")(x)
    
    if model_name == 'irrelevant_vs_relevant'
        out = Dense(1, activation="sigmoid", name='irrelevant')(x)
    elif model_name == 'multitask':
        out_width = Dense(1, activation="sigmoid", name='single_car')(x)
        out_pavement = Dense(3, activation="softmax", name='pavement')(x)
    elif model_name == 'quality':
        out = Dense(3, activation='softmax', name='quality')
        
    # TODO: Para o multitask, tens de fazer duas losses
    """
    loss = {
        'single_car': binary_crossentropy,
        'pavement': categorical_crossentropy
    }
    """

    model = Model(input_layer, out)
    optimizer = Adam(learning_rate=lr)
    model.compile(optimizer, "binary_crossentropy", ["accuracy"])

    return model